In [42]:
import numpy as np
from tensorflow import keras
# import keras
from keras.layers.wrappers import Bidirectional
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
import keras
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout
import pandas as pd

In [84]:
model_tte = Sequential()
model_tte.add(LSTM(units=64, input_shape=(168, 50), return_sequences=True))
model_tte.add(Dropout(0.3))
model_tte.add(LSTM(units=32))
model_tte.add(Dropout(0.2))
model_tte.add(Dense(units=10, activation='relu'))
model_tte.add(Dense(units=6, activation='softmax'))

model_tte.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [64]:
def intialize_emb_matrix(file):
    embedding_matrix = {}
    for line in file:
        values = line.split()
        word = values[0]
        embedding = np.array(values[1:], dtype='float64')
        embedding_matrix[word] = embedding

    return embedding_matrix 

def get_emb_data(data, max_len, file):
#     max_len = 168
    embedding_matrix = intialize_emb_matrix(file)
    embedding_data = np.zeros((len(data), max_len, 50))  # from glove6B50d
    
    for idx in range(data.shape[0]):
        words_in_sentence = data[idx].split()
        
        for i in range(len(words_in_sentence)):
            if embedding_matrix.get(words_in_sentence[i].lower()) is not None:
                embedding_data[idx][i] = embedding_matrix[words_in_sentence[i].lower()]
                
    return embedding_data

In [85]:
file = open('drive/MyDrive/emoji/glove.6B.50d.txt',encoding='utf8')
data = pd.read_csv('drive/MyDrive/emoji/twitter_dataset.csv',sep=',')
X_train, y_train = data['tweet'].values, data['emoji'].values

le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

print(X_train.shape,y_train.shape)


X_temb = get_emb_data(X_train, 168, file)
y_train = to_categorical(y_train)

(793,) (793,)


In [81]:
print(X_temb.shape,y_train.shape)

(793, 100, 50) (793, 6)


In [83]:

res = model_tte.fit(X_temb, y_train, validation_split=0.2, batch_size=128, epochs=20, verbose=2)

Epoch 1/20
5/5 - 6s - loss: 1.7895 - acc: 0.2129 - val_loss: 1.8370 - val_acc: 0.0000e+00 - 6s/epoch - 1s/step
Epoch 2/20
5/5 - 1s - loss: 1.7811 - acc: 0.2192 - val_loss: 1.9195 - val_acc: 0.0000e+00 - 1s/epoch - 204ms/step
Epoch 3/20
5/5 - 1s - loss: 1.7675 - acc: 0.2240 - val_loss: 2.1639 - val_acc: 0.0000e+00 - 1s/epoch - 209ms/step
Epoch 4/20
5/5 - 1s - loss: 1.7392 - acc: 0.2208 - val_loss: 2.7362 - val_acc: 0.0000e+00 - 1s/epoch - 204ms/step
Epoch 5/20
5/5 - 1s - loss: 1.7344 - acc: 0.2303 - val_loss: 2.8644 - val_acc: 0.0000e+00 - 1s/epoch - 212ms/step
Epoch 6/20
5/5 - 1s - loss: 1.7387 - acc: 0.2224 - val_loss: 2.7464 - val_acc: 0.0000e+00 - 1s/epoch - 201ms/step
Epoch 7/20
5/5 - 1s - loss: 1.7299 - acc: 0.2271 - val_loss: 2.6846 - val_acc: 0.0000e+00 - 998ms/epoch - 200ms/step
Epoch 8/20
5/5 - 1s - loss: 1.7279 - acc: 0.2224 - val_loss: 2.8078 - val_acc: 0.0000e+00 - 1s/epoch - 203ms/step
Epoch 9/20
5/5 - 1s - loss: 1.7224 - acc: 0.2066 - val_loss: 3.0147 - val_acc: 0.0000e+0